In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import csv
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield instance

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# ONE RUN
#############################################

agent = simulation.simulator(
                     connection_label = "lonely_worker",
                     q_network_type = 'simple',
                     target_q_network_type = 'simple',
                     gamma = 0.99,
                     target_update_freq = 10000,
                     train_freq = 3,
                     num_burn_in = 300,
                     batch_size = 32,
                     optimizer = optimizers.RMSprop(lr=0.001, rho=0.95),
                     loss_func = "mse",
                     max_ep_length = 1000,
                     experiment_id = "Exp_1",
                     model_checkpoint = True,
                     opt_metric = None,
                     
                    # environment parameters
                     net_file = "./network/cross.net.xml",
                     route_file = "./network/cross.rou.xml",
                     state_shape = (1,11),
                     num_actions = 2,
                     use_gui = False,
                     delta_time = 10,
                 
                    # memory parameters
                     max_size = 100000,
                 
                    # additional parameters
                 
                     policy = "linDecEpsGreedy",
                     eps = 0.1,
                     num_episodes = 2,
                     monitoring = True)
agent.train(3)
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")
agent.evaluate(runs=1, use_gui= True)

FatalTraCIError: connection closed by SUMO

In [ ]:
agent.evaluate(runs=1, use_gui= True)

In [ ]:
# GRIDSEARCH
###############################################################

exp_id = "Test _la"

param = {
    "batch_size" : [30],
    "target_update_frequency" : [5000, 10000],#, 20000,40000]
    "gamma" : [0.9, 0.95],#,0.99]
    "train_freq" : [1,3],
    "max_size" : [20000, 100000]
}

param_grid = iter_params(**param)

def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all objects
    
    print('Run', position + 1, '-- parameters', args)
    
    agent = simulation.simulator(connection_label = position+1,
                                batch_size = args[0],
                                target_update_freq = args[1],
                                gamma = args[2],
                                train_freq = args[3],
                                route_file = "cross.rou.xml",
                                max_size = args[4],
                                monitoring = True,
                                num_episodes =  1,
                                experiment_id = exp_id)
    
    agent.train()
    result = agent.evaluate(runs = 5)

    return (multiprocessing.current_process().name, position + 1, args, result)


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        # print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
    
    
    with open(os.path.join("./logs",exp_id,"GS_results"), "w",newline='') as file:
        writer = csv.writer(file, dialect = 'excel')
        writer.writerow(["Process", "Run", "Parameters", "Result"])
    
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",exp_id,"GS_results"), "a",newline='') as file:
            writer = csv.writer(file, dialect = 'excel')
            writer.writerow(results[-1])                
        print('%s -- run %s -- parameters %s -- result %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)